# 1) Numerical Check for undamped duffing oscillator

## Initial Chek at t=0

In [4]:
import numpy as np

def track_mismatch(I, omega_init):
    """
    Tracks any mismatch between LHS_i(0) and K_i for i=1,2,3 
    right at t=0, before any numeric integration. This identifies 
    constant offsets or sign errors in (A_i,B_i,K_i).
    
    Parameters
    ----------
    I : [I1, I2, I3]
        Principal moments of inertia.
    omega_init : [omega1(0), omega2(0), omega3(0)]
        Initial angular velocities.
    
    Prints the computed (A_i, B_i, K_i) for each axis, 
    then compares LHS_i(0) to K_i.
    """
    I1, I2, I3 = I
    w1_0, w2_0, w3_0 = omega_init

    w1_dot_0 = (-(I3-I2) * w2_0 * w3_0) / I1
    w2_dot_0 = (-(I1-I3) * w3_0 * w1_0) / I2
    w3_dot_0 = (-(I2-I1) * w1_0 * w2_0) / I3

    # 1) Compute H^2, T
    H_sq = (I1*w1_0)**2 + (I2*w2_0)**2 + (I3*w3_0)**2
    T    = 0.5*((I1*w1_0**2) + (I2*w2_0**2) + (I3*w3_0**2))

    # 2) Compute A_i, B_i
    A1 = ((I1 - I2)*(2*I3*T - H_sq) + (I1 - I3)*(2*I2*T - H_sq)) / (I1*I2*I3)
    A2 = ((I2 - I3)*(2*I1*T - H_sq) + (I2 - I1)*(2*I3*T - H_sq)) / (I1*I2*I3)
    A3 = ((I3 - I1)*(2*I2*T - H_sq) + (I3 - I2)*(2*I1*T - H_sq)) / (I1*I2*I3)

    B1 = (2*(I1 - I2)*(I1 - I3)) / (I2*I3)
    B2 = (2*(I2 - I1)*(I2 - I3)) / (I1*I3)
    B3 = (2*(I3 - I1)*(I3 - I2)) / (I1*I2)

    # 3) Compute K_i
    K1 = ((2*I2*T - H_sq)*(H_sq - 2*I3*T)) / ((I1**2)*I2*I3)
    K2 = ((2*I3*T - H_sq)*(H_sq - 2*I1*T)) / (I1*(I2**2)*I3)
    K3 = ((2*I1*T - H_sq)*(H_sq - 2*I2*T)) / (I1*I2*(I3**2))

    print("=== Inertia, H^2, T ===")
    print(f"I1={I1}, I2={I2}, I3={I3}")
    print(f"omega_init=({w1_0}, {w2_0}, {w3_0})")
    print(f"H_sq={H_sq:.6f}, T={T:.6f}")
    print("\n=== Computed (A_i, B_i, K_i) ===")
    print(f"A1={A1:.6f}, A2={A2:.6f}, A3={A3:.6f}")
    print(f"B1={B1:.6f}, B2={B2:.6f}, B3={B3:.6f}")
    print(f"K1={K1:+.6f}, K2={K2:+.6f}, K3={K3:+.6f}\n")

    # 4) Include dot_omega_i(0) in LHS calculation
    def lhs_i(dot_w0, Ai, Bi, w0):
        return dot_w0**2 + Ai * (w0**2) + 0.5 * Bi * (w0**4)
    
    lhs1_0 = lhs_i(w1_dot_0, A1, B1, w1_0)
    lhs2_0 = lhs_i(w2_dot_0, A2, B2, w2_0)
    lhs3_0 = lhs_i(w3_dot_0, A3, B3, w3_0)

    diff1 = lhs1_0 - K1
    diff2 = lhs2_0 - K2
    diff3 = lhs3_0 - K3

    print("=== Checking LHS_i(0) - K_i ===\n")
    print(f"Axis1: LHS1(0)={lhs1_0:+.6f},  K1={K1:+.6f},  LHS1-K1={diff1:+.12f}")
    print(f"Axis2: LHS2(0)={lhs2_0:+.6f},  K2={K2:+.6f},  LHS2-K2={diff2:+.12f}")
    print(f"Axis3: LHS3(0)={lhs3_0:+.6f},  K3={K3:+.6f},  LHS3-K3={diff3:+.12f}")

    print("\nIf these differences are nonzero at t=0, there is a sign/factor mismatch.")
    print("Check each formula carefully or reorder (I_1,I_2,I_3) if needed.\n")

# ---------------
# Example usage:
# ---------------
if __name__ == "__main__":
    I_vals = [5.0, 3, 2.0]
    omega_init = [1.0, 2, 1]
    # Assuming initial derivatives (needs to be provided or computed)
    track_mismatch(I_vals, omega_init)


=== Inertia, H^2, T ===
I1=5.0, I2=3, I3=2.0
omega_init=(1.0, 2, 1)
H_sq=65.000000, T=9.500000

=== Computed (A_i, B_i, K_i) ===
A1=-2.600000, A2=2.800000, A3=-0.200000
B1=2.000000, B2=-0.400000, B3=0.400000
K1=-1.440000, K2=+9.000000, K3=+4.000000

=== Checking LHS_i(0) - K_i ===

Axis1: LHS1(0)=-1.440000,  K1=-1.440000,  LHS1-K1=+0.000000000000
Axis2: LHS2(0)=+9.000000,  K2=+9.000000,  LHS2-K2=+0.000000000000
Axis3: LHS3(0)=+4.000000,  K3=+4.000000,  LHS3-K3=+0.000000000000

If these differences are nonzero at t=0, there is a sign/factor mismatch.
Check each formula carefully or reorder (I_1,I_2,I_3) if needed.



## Propagating over specified time period (worked with DeepSeek)

In [8]:
import numpy as np

def compute_derivatives(I, omega):
    """Compute angular accelerations using Euler's equations for torque-free motion."""
    I1, I2, I3 = I
    w1, w2, w3 = omega
    w1_dot = ((I2 - I3) * w2 * w3) / I1
    w2_dot = ((I3 - I1) * w3 * w1) / I2
    w3_dot = ((I1 - I2) * w1 * w2) / I3
    return np.array([w1_dot, w2_dot, w3_dot])

def rk4_step(I, omega, dt):
    """Perform one RK4 integration step."""
    k1 = compute_derivatives(I, omega)
    k2 = compute_derivatives(I, omega + 0.5*dt*k1)
    k3 = compute_derivatives(I, omega + 0.5*dt*k2)
    k4 = compute_derivatives(I, omega + dt*k3)
    omega_new = omega + dt * (k1 + 2*k2 + 2*k3 + k4) / 6
    return omega_new

def track_mismatch_over_time(I, omega_init, t_end, dt, check_interval=0.5):
    """
    Simulate torque-free motion and check LHS vs K_i at specified intervals.
    
    Parameters
    ----------
    I : [I1, I2, I3]
        Principal moments of inertia.
    omega_init : [omega1(0), omega2(0), omega3(0)]
        Initial angular velocities.
    t_end : float
        Total simulation time.
    dt : float
        Time step for numerical integration.
    check_interval : float
        Interval at which to print LHS vs K_i checks.
    """
    I1, I2, I3 = I
    omega = np.array(omega_init, dtype=float)
    t = 0.0

    # Precompute constants (T, H_sq, A_i, B_i, K_i) once at t=0
    H_sq = (I1*omega[0])**2 + (I2*omega[1])**2 + (I3*omega[2])**2
    T = 0.5 * (I1*omega[0]**2 + I2*omega[1]**2 + I3*omega[2]**2)
    
    A1 = ((I1 - I2)*(2*I3*T - H_sq) + (I1 - I3)*(2*I2*T - H_sq)) / (I1*I2*I3)
    A2 = ((I2 - I3)*(2*I1*T - H_sq) + (I2 - I1)*(2*I3*T - H_sq)) / (I1*I2*I3)
    A3 = ((I3 - I1)*(2*I2*T - H_sq) + (I3 - I2)*(2*I1*T - H_sq)) / (I1*I2*I3)
    
    B1 = (2*(I1 - I2)*(I1 - I3)) / (I2*I3)
    B2 = (2*(I2 - I1)*(I2 - I3)) / (I1*I3)
    B3 = (2*(I3 - I1)*(I3 - I2)) / (I1*I2)
    
    K1 = ((2*I2*T - H_sq)*(H_sq - 2*I3*T)) / (I1**2 * I2 * I3)
    K2 = ((2*I3*T - H_sq)*(H_sq - 2*I1*T)) / (I1 * I2**2 * I3)
    K3 = ((2*I1*T - H_sq)*(H_sq - 2*I2*T)) / (I1 * I2 * I3**2)

    print("=== Precomputed Constants (T, H^2, A_i, B_i, K_i) ===")
    print(f"T = {T:.6f}, H^2 = {H_sq:.6f}\n")
    print(f"A1 = {A1:.6f}, A2 = {A2:.6f}, A3 = {A3:.6f}")
    print(f"B1 = {B1:.6f}, B2 = {B2:.6f}, B3 = {B3:.6f}")
    print(f"K1 = {K1:+.6f}, K2 = {K2:+.6f}, K3 = {K3:+.6f}\n")

    # Store checkpoints (e.g., t=0.0, 0.5, 1.0, ...)
    next_check_time = 0.0

    while t <= t_end:
        if abs(t - next_check_time) < 1e-6:
            # Compute current omega_dot
            omega_dot = compute_derivatives(I, omega)
            # Calculate LHS for each axis
            lhs1 = omega_dot[0]**2 + A1 * omega[0]**2 + 0.5 * B1 * omega[0]**4
            lhs2 = omega_dot[1]**2 + A2 * omega[1]**2 + 0.5 * B2 * omega[1]**4
            lhs3 = omega_dot[2]**2 + A3 * omega[2]**2 + 0.5 * B3 * omega[2]**4
            
            # Print results
            print(f"=== Check at t = {t:.2f} ===")
            print(f"omega = [{omega[0]:.6f}, {omega[1]:.6f}, {omega[2]:.6f}]")
            print(f"omega_dot = [{omega_dot[0]:.6f}, {omega_dot[1]:.6f}, {omega_dot[2]:.6f}]")
            print(f"LHS1: {lhs1:.12f}")
            print(f"LHS2: {lhs2:.12f}")
            print(f"LHS3: {lhs3:.12f}")
            print(f"LHS1-K1 = {lhs1 - K1:+.12e}")
            print(f"LHS2-K2 = {lhs2 - K2:+.12e}")
            print(f"LHS3-K3 = {lhs3 - K3:+.12e}\n")
            next_check_time += check_interval

        # Integrate to next time step
        omega = rk4_step(I, omega, dt)
        t += dt

# Example usage
if __name__ == "__main__":
    I_vals = [5.0, 3.0, 2.0]
    omega_init = [1.0, 2.0, 1.0]
    track_mismatch_over_time(I_vals, omega_init, t_end=20, dt=0.001, check_interval=1)

=== Precomputed Constants (T, H^2, A_i, B_i, K_i) ===
T = 9.500000, H^2 = 65.000000

A1 = -2.600000, A2 = 2.800000, A3 = -0.200000
B1 = 2.000000, B2 = -0.400000, B3 = 0.400000
K1 = -1.440000, K2 = +9.000000, K3 = +4.000000

=== Check at t = 0.00 ===
omega = [1.000000, 2.000000, 1.000000]
omega_dot = [0.400000, -1.000000, 2.000000]
LHS1: -1.440000000000
LHS2: 9.000000000000
LHS3: 4.000000000000
LHS1-K1 = +0.000000000000e+00
LHS2-K2 = +0.000000000000e+00
LHS3-K3 = +0.000000000000e+00

=== Check at t = 1.00 ===
omega = [1.337471, -0.236334, 2.223544]
omega_dot = [-0.105100, -2.973926, -0.316090]
LHS1: -1.440000000000
LHS2: 9.000000000000
LHS3: 4.000000000000
LHS1-K1 = +2.708944180085e-14
LHS2-K2 = +1.652011860642e-13
LHS3-K3 = +2.930988785010e-14

=== Check at t = 2.00 ===
omega = [0.945080, -2.129346, 0.682557]
omega_dot = [-0.290680, -0.645072, -2.012403]
LHS1: -1.440000000000
LHS2: 9.000000000000
LHS3: 4.000000000000
LHS1-K1 = +4.440892098501e-15
LHS2-K2 = -7.105427357601e-15
LHS3-K3 =

## Propagating over time (worked with o1 model)

In [11]:
import numpy as np
from scipy.integrate import solve_ivp

def torque_free(t, w, I):
    """
    Computes d(omega)/dt for torque-free Euler's equations in principal axes.

    Parameters
    ----------
    t : float
        Current time (not used for an autonomous system).
    w : array-like of shape (3,)
        Current angular velocities: [w1, w2, w3].
    I : array-like of shape (3,)
        Principal moments of inertia: [I1, I2, I3].

    Returns
    -------
    dw : list of length 3
        The time derivatives [dw1/dt, dw2/dt, dw3/dt].
    """
    I1, I2, I3 = I
    w1, w2, w3 = w
    dw1 = ((I2 - I3) * w2 * w3) / I1
    dw2 = ((I3 - I1) * w3 * w1) / I2
    dw3 = ((I1 - I2) * w1 * w2) / I3
    return [dw1, dw2, dw3]

def track_mismatch_solve_ivp(I, omega_init, t_end=20.0, dt=0.001, check_interval=1.0):
    """
    Integrate torque-free rotation using solve_ivp, then check Duffing-like LHS vs. K_i.

    Parameters
    ----------
    I : [I1, I2, I3]
        Principal moments of inertia.
    omega_init : [w1, w2, w3]
        Initial angular velocities.
    t_end : float
        End time for integration.
    dt : float
        Step size for storing solution in time array.
    check_interval : float
        Print checks each time t is a multiple of this interval.
    """
    I1, I2, I3 = I
    w1_0, w2_0, w3_0 = omega_init

    # ----------------------------------------------------------------
    # 1) Precompute the "Duffing-like" constants A_i, B_i, K_i
    #    using T, H^2, the same approach as before.
    # ----------------------------------------------------------------
    H_sq = (I1*w1_0)**2 + (I2*w2_0)**2 + (I3*w3_0)**2
    T = 0.5 * (I1*(w1_0**2) + I2*(w2_0**2) + I3*(w3_0**2))

    A1 = ((I1 - I2)*(2*I3*T - H_sq) + (I1 - I3)*(2*I2*T - H_sq)) / (I1*I2*I3)
    A2 = ((I2 - I3)*(2*I1*T - H_sq) + (I2 - I1)*(2*I3*T - H_sq)) / (I1*I2*I3)
    A3 = ((I3 - I1)*(2*I2*T - H_sq) + (I3 - I2)*(2*I1*T - H_sq)) / (I1*I2*I3)

    B1 = (2*(I1 - I2)*(I1 - I3)) / (I2*I3)
    B2 = (2*(I2 - I1)*(I2 - I3)) / (I1*I3)
    B3 = (2*(I3 - I1)*(I3 - I2)) / (I1*I2)

    K1 = ((2*I2*T - H_sq)*(H_sq - 2*I3*T)) / (I1**2*I2*I3)
    K2 = ((2*I3*T - H_sq)*(H_sq - 2*I1*T)) / (I1*I2**2*I3)
    K3 = ((2*I1*T - H_sq)*(H_sq - 2*I2*T)) / (I1*I2*I3**2)

    print("=== Precomputed Constants (T, H^2, A_i, B_i, K_i) ===")
    print(f"T    = {T:.6f}, H^2 = {H_sq:.6f}")
    print(f"A1 = {A1:.6f}, A2 = {A2:.6f}, A3 = {A3:.6f}")
    print(f"B1 = {B1:.6f}, B2 = {B2:.6f}, B3 = {B3:.6f}")
    print(f"K1 = {K1:+.6f}, K2 = {K2:+.6f}, K3 = {K3:+.6f}\n")

    # ----------------------------------------------------------------
    # 2) Integrate using solve_ivp over [0, t_end] 
    #    with times in steps of dt.
    # ----------------------------------------------------------------
    t_eval = np.arange(0, t_end+dt, dt)

    sol = solve_ivp(
        fun=lambda t, y: torque_free(t, y, I),
        t_span=(0.0, t_end),
        y0=omega_init,
        t_eval=t_eval,
        method='RK45',
        rtol=1e-12,
        atol=1e-12
    )

    # sol.t -> array of times (matches t_eval)
    # sol.y -> shape (3, len(t_eval)), i.e. [w1(t), w2(t), w3(t)] in rows.

    # ----------------------------------------------------------------
    # 3) Post-process to check LHS_i(t) - K_i at each time step
    #    We'll only print when t is a multiple of check_interval 
    #    (within a small tolerance).
    # ----------------------------------------------------------------
    t_array = sol.t
    w_array = sol.y.T  # shape (len(t_eval), 3)

    next_print_time = 0.0
    small_tol = 1e-10

    for i, t_now in enumerate(t_array):
        w1, w2, w3 = w_array[i]

        # If we're close to a multiple of check_interval, do the check
        if abs(t_now - next_print_time) < small_tol:
            # Compute w_dot from the torque-free equation at this (t, w)
            w_dot = torque_free(t_now, [w1, w2, w3], I)
            # w_dot is [dw1/dt, dw2/dt, dw3/dt]

            # LHS for each axis: w_dot_i^2 + A_i*w_i^2 + 0.5*B_i*w_i^4
            lhs1 = w_dot[0]**2 + A1*(w1**2) + 0.5*B1*(w1**4)
            lhs2 = w_dot[1]**2 + A2*(w2**2) + 0.5*B2*(w2**4)
            lhs3 = w_dot[2]**2 + A3*(w3**2) + 0.5*B3*(w3**4)

            diff1 = lhs1 - K1
            diff2 = lhs2 - K2
            diff3 = lhs3 - K3

            print(f"=== Check at t = {t_now:.3f} ===")
            print(f"omega  = [{w1:.6f}, {w2:.6f}, {w3:.6f}]")
            print(f"dw/dt = [{w_dot[0]:.6f}, {w_dot[1]:.6f}, {w_dot[2]:.6f}]")
            print(f"LHS1   = {lhs1:+.12e},   LHS1-K1 = {diff1:+.12e}")
            print(f"LHS2   = {lhs2:+.12e},   LHS2-K2 = {diff2:+.12e}")
            print(f"LHS3   = {lhs3:+.12e},   LHS3-K3 = {diff3:+.12e}\n")

            next_print_time += check_interval

    print("...Done.\n")

# ------------------
# Example usage
# ------------------
if __name__ == "__main__":
    # Example principal moments
    I_vals = [5.0, 3.0, 2.0]
    # Suppose we have initial angular velocities with nonzero derivatives from Euler eqns
    # (the system will figure out w_dot from torque_free function).
    omega_init = [0.3, 0.8, 0.5]

    # Integrate up to t=20s, storing steps of dt=0.001, 
    # printing the mismatch every 1 second
    track_mismatch_solve_ivp(I_vals, omega_init, t_end=20, dt=0.001, check_interval=0.1)


=== Precomputed Constants (T, H^2, A_i, B_i, K_i) ===
T    = 1.435000, H^2 = 9.010000
A1 = -0.258000, A2 = 0.396000, A3 = -0.138000
B1 = 2.000000, B2 = -0.400000, B3 = 0.400000
K1 = -0.008720, K2 = +0.194020, K3 = +0.035600

=== Check at t = 0.000 ===
omega  = [0.300000, 0.800000, 0.500000]
dw/dt = [0.080000, -0.150000, 0.240000]
LHS1   = -8.720000000000e-03,   LHS1-K1 = +2.428612866368e-17
LHS2   = +1.940200000000e-01,   LHS2-K2 = +2.775557561563e-17
LHS3   = +3.560000000000e-02,   LHS3-K3 = -1.387778780781e-16

=== Check at t = 0.100 ===
omega  = [0.308113, 0.784431, 0.524088]
dw/dt = [0.082222, -0.161478, 0.241693]
LHS1   = -8.719999999982e-03,   LHS1-K1 = +1.764560719764e-14
LHS2   = +1.940200000002e-01,   LHS2-K2 = +1.902089596939e-13
LHS3   = +3.560000000010e-02,   LHS3-K3 = +1.020433737509e-13

=== Check at t = 0.200 ===
omega  = [0.316436, 0.767686, 0.548323]
dw/dt = [0.084188, -0.173509, 0.242923]
LHS1   = -8.719999999985e-03,   LHS1-K1 = +1.527250548250e-14
LHS2   = +1.940200